# Tutorial

`micromodelsim` enables the simulation of diffusion-weighted nuclear magnetic resonance (NMR) measurements using microstructural models. This minimal tutorial covers its basic functionality, and we recommend the reader to also look at [the reference](https://micromodelsim.rtfd.io/en/latest/reference.html) where essential details are provided.

Let's start by importing the required packages.


In [1]:
import os

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import micromodelsim as mmsim

## Gradients

Gradient information is stored in a `Gradient` object. The code below defines and visualizes a two-shell acquisition protocol.

In [ ]:
bvecs = np.array(
    [
        [0.283, 0.283, 0.917],
        [-0.283, 0.283, 0.917],
        [-0.283, -0.283, 0.917],
        [0.283, -0.283, 0.917],
        [0.689, 0.285, 0.667],
        [0.285, 0.689, 0.667],
        [-0.285, 0.689, 0.667],
        [-0.689, 0.285, 0.667],
        [-0.689, -0.285, 0.667],
        [-0.285, -0.689, 0.667],
        [0.285, -0.689, 0.667],
        [0.689, -0.285, 0.667],
        [0.943, 0.0, 0.333],
        [0.667, 0.667, 0.333],
        [0.0, 0.943, 0.333],
        [-0.667, 0.667, 0.333],
        [-0.943, 0.0, 0.333],
        [-0.667, -0.667, 0.333],
        [-0.0, -0.943, 0.333],
        [0.667, -0.667, 0.333],
        [0.924, 0.383, 0.0],
        [0.383, 0.924, 0.0],
        [-0.383, 0.924, 0.0],
        [-0.924, 0.383, 0.0],
    ]
)

bvecs = np.concatenate((bvecs, -bvecs))

bvals = np.concatenate((np.ones(len(bvecs)), 2 * np.ones(len(bvecs))))

bvecs = np.vstack((bvecs, bvecs))

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(bvals * bvecs[:, 0], bvals * bvecs[:, 1], bvals * bvecs[:, 2])
ax.set_axis_off()
fig.tight_layout()
plt.show()

gradient = mmsim.Gradient(bvals, bvecs)

## Diffusion tensor distribution simulation

The function `dtd_simulation` enables simulating measurements from a diffusion tensor distribution (DTD). The signal is given by the following equation:

$$S = \sum_{i=1}^N f_i \exp \left( -\mathbf{b}:\mathbf{D}_i \right)$$

where $N$ is the number of diffusion tensors, $f_i$ is a signal fraction, $\mathbf{b}$ is the b-tensor, $\mathbf{D}_i$ is a diffusion tensor, and $:$ denotes the generalized scalar product.

The code below generates signals from a simple DTD using the acquisition protocol defined above.

In [ ]:
dtd = np.array(
    [
        [[2, 0, 0], [0, 0, 0], [0, 0, 0]],
        [[2, 0, 0], [0, 0.5, 0], [0, 0, 0.5]],
        [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    ]
)  # the array has to have shape (number of tensors, 3, 3)

signals = mmsim.dtd_simulation(gradient, dtd)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
im = ax.scatter(
    bvals * bvecs[:, 0],
    bvals * bvecs[:, 1],
    bvals * bvecs[:, 2],
    c=signals,
    cmap="turbo",
)
plt.colorbar(im)
ax.set_title("Signals")
ax.set_axis_off()
fig.tight_layout()
plt.show()

The function `add_noise` allows random noise to be added to the simulated signals. Random number generator seed can be set with `numpy.random.seed`.

In [ ]:
snr = 20  # signal-to-noise ratio
noisy_signals = mmsim.add_noise(signals, snr)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
im = ax.scatter(
    bvals * bvecs[:, 0],
    bvals * bvecs[:, 1],
    bvals * bvecs[:, 2],
    c=noisy_signals,
    cmap="turbo",
)
plt.colorbar(im)
ax.set_title("Noisy signals")
ax.set_axis_off()
fig.tight_layout()
plt.show()

## Compartment model simulation

The function `compartment_model_simulation` enables simulating measurements from a Gaussian compartment model and an orientation distribution function (ODF).

The signal is expressed as a spherical convolution of the ODF by a microstructural kernel response function $K$:

$$S(\mathbf{x}) = \int_{\mathbf{R} \in \text{SO}(3)} \text{d}\mathbf{R} \ \text{ODF}(\mathbf{R} \mathbf{\hat{e}_3}) K(\mathbf{R}^{-1} \mathbf{x})$$

where $\mathbf{x} \in S^2$ is the diffusion encoding direction and

$$K(\mathbf{\hat{n}}) = \sum_{i=1}^N f_i \exp(- \mathbf{b}:\mathbf{D}_i)$$

where $N$ is the number of compartments, $f_i$ is a signal fraction, $\mathbf{b}$ is the b-tensor corresponding to $\mathbf{x}$ and a b-value equal to $\text{Tr}(\mathbf{b})$, $:$ denotes the generalized scalar product, and $\mathbf{D}_i$ is a diffusion tensor aligned with the $z$-axis.

### Spherical harmonics

ODFs must be represented as spherical harmonics expansions. Spherical harmonics can be understood as a Fourier basis on the sphere. Because reconstructed diffusion MRI signals are real-valued and antipodally symmetric, the following basis is used:

$$S_l^m = \left\{\begin{matrix} 0 & \text{if } l \text{ is odd} \\ \sqrt{2} \ \Im \left( Y_l^{-m} \right) & \text{if} \ m < 0 \\  Y_l^0 & \text{if} \ m = 0 \\ \sqrt{2} \ \Re \left( Y_l^{m} \right) & \text{if} \ m > 0  \end{matrix}\right. $$

where $l$ is the degree, $m$ is the order, and $Y_l^m$ is a spherical harmonic:

$$Y_l^m(\theta, \phi) = \sqrt{\frac{2l + 1}{4 \pi} \frac{(l - m)!} {(l + m)!}} P_l^m(\cos \theta) e^{im\phi}$$

where $\theta \in [0, \pi]$ is the polar angle and $\phi \in [0, 2\pi)$ is the azimuthal angle.

The code below visualizes the basis functions until $l = 8$.

In [ ]:
N = 100  # determines the resolution (higher resolution takes longer to render)

mg_thetas, mg_phis = np.meshgrid(
    np.linspace(0, np.pi, N), np.linspace(0, 2 * np.pi, 2 * N)
)

fig = plt.figure(figsize=(17, 5))
l_max = 8
for l in range(0, l_max + 1, 2):
    c = int(l / 2 * 17) + l_max - l
    for m in range(-l, l + 1):
        c += 1
        ax = fig.add_subplot(5, 17, c, projection="3d")
        values = mmsim.sh(l, m, mg_thetas, mg_phis)
        if l == 0:
            fcolors = np.zeros(values.shape)
        else:
            fcolors = (values - np.min(values)) / (np.max(values) - np.min(values))
        ax.plot_surface(
            np.sin(mg_thetas) * np.cos(mg_phis),
            np.sin(mg_thetas) * np.sin(mg_phis),
            np.cos(mg_thetas),
            facecolors=matplotlib.cm.turbo(fcolors),
            rcount=2 * N,
            ccount=N,
            shade=False,
        )
        ax.set_axis_off()
fig.tight_layout()
plt.show()

ODFs must be represented as a 1D arrays using the following indexing:

$$i = \frac{1}{2}l(l + 1) + m$$

The number of coefficients $N$ depends on $l_\text{max}$:

$$N = \frac{1}{2}(l_\text{max} + 1)(l_\text{max} + 2)$$

The ODFs don't have to be normalized as the `compartment_model_simulation` function does the normalization.

The code below defines and visualizes a spherical harmonics expansion with $l_\text{max} = 8$.

In [ ]:
l_max = 8
n_coeffs = int(0.5 * (l_max + 1) * (l_max + 2))

_l0s = np.zeros(n_coeffs)
for l in range(0, l_max + 1, 2):
    for m in range(-l, l + 1):
        _l0s[int(0.5 * l * (l + 1) + m)] = int(0.5 * l * (l + 1))
coeffs = np.ones(n_coeffs)
np.random.seed(666)
coeffs[1::] = 1 / _l0s[1::] * (0.5 - np.random.random(n_coeffs - 1))

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(np.arange(n_coeffs), coeffs)
ax.set_xlabel("i")
ax.set_ylabel("Expansion coefficient")
plt.show()

values = np.zeros(mg_thetas.shape)
for l in range(0, l_max + 1, 2):
    for m in range(-l, l + 1):
        i = int(0.5 * l * (l + 1) + m)
        values += coeffs[i] * mmsim.sh(l, m, mg_thetas, mg_phis)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
fcolors = (values - np.min(values)) / (np.max(values) - np.min(values))
im = ax.plot_surface(
    np.sin(mg_thetas) * np.cos(mg_phis),
    np.sin(mg_thetas) * np.sin(mg_phis),
    np.cos(mg_thetas),
    facecolors=matplotlib.cm.turbo(fcolors),
    rcount=2 * N,
    ccount=N,
    shade=False,
)
ax.set_axis_off()
plt.show()

### Signal generation

Now that we have an ODF, we can define the compartment signal fractions, axial diffusivities, and radial diffusivites, and generate synthetic signals:

In [ ]:
fs = np.array([0.8, 0.2])[np.newaxis]
ads = np.array([2, 0])[np.newaxis]
rds = np.array([1, 1])[np.newaxis]
odfs_sh = coeffs[np.newaxis]
signals = mmsim.compartment_model_simulation(gradient, fs, ads, rds, odfs_sh)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
im = ax.scatter(
    bvals * bvecs[:, 0],
    bvals * bvecs[:, 1],
    bvals * bvecs[:, 2],
    c=signals,
    cmap="turbo",
)
plt.colorbar(im)
ax.set_axis_off()
fig.tight_layout()
plt.show()